# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

### Q: since we have all salary data, why do hypothesis testing?

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
print(salaries.shape)
salaries.isnull().sum()

(33183, 8)


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
hourly_cnt = salaries['Hourly Rate'].notnull().sum()
salaried_cnt = salaries['Annual Salary'].notnull().sum()
print(hourly_cnt)
print(salaried_cnt)

# just to check if there is a data missmatch -> but it's either Annual or Hourly Rate for all records
salaries[(salaries['Annual Salary'].isnull()) & (salaries['Hourly Rate'].isnull())]

8022
25161


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
salaries.pivot_table(index=['Department'], values=['Name'], aggfunc = ['count'])

,count
,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# Your code here:
import scipy.stats as st

hourly = salaries['Hourly Rate'][(salaries['Hourly Rate'].notnull())]
print(hourly.shape)

# claim: hourly wage of all hourly workers is significantly different from $30/hr.
# H0: mu = 30
mu_test = 30
c = 0.95

# manual calculation
m = np.mean(hourly)
s = np.std(hourly, ddof=1) # we could also take ddof=0 since we have the whole population?
n = len(hourly)
stat = (m-mu_test)/(s/np.sqrt(n))
print(stat)
# p-value 2-tailed
p = st.t.sf(abs(stat),n-1)*2
display(p)

# scipy function shortcut
display(st.ttest_1samp(hourly,mu_test))

# compare p with c
if p < (1-c):
    print('H0 is rejected, and therefore hourly wage of all hourly workes differs signaficatantly from $30/hr')
else:
    print('H0 can not be rejected!')

(8022,)
20.6198057854942


4.3230240486229894e-92

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

H0 is rejected, and therefore hourly wage of all hourly workes differs signaficatantly from $30/hr


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [8]:
# Your code here:
# H0: mu <= 86000

salaried_police = salaries['Annual Salary'][(salaries['Annual Salary'].notnull()) & (salaries['Department'] == 'POLICE')]
print(salaried_police.shape)

mu_test = 86000
c = 0.95
m = np.mean(salaried_police)
s = np.std(salaried_police, ddof=1)
n = len(salaried_police)
stat = (m-mu_test)/(s/np.sqrt(n))
print(stat)
# p-value 1-tailed
p = st.t.sf(abs(stat),n-1)
display(p)

# scipy function shortcut
display(st.ttest_1samp(salaried_police,mu_test))

# compare p with c
if (p < (1-c)) & (stat > 0):
    print('H0 is rejected, and therefore one can claim with 95% confidence that all salaries this year are higher than last year\'s mean of 86K USD')
else:
    print('H0 can not be rejected!')

(13404,)
3.081997005712994


0.0010301701775482569

Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.0020603403550965137)

H0 is rejected, and therefore one can claim with 95% confidence that all salaries this year are higher than last year's mean of 86K USD


Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
# Your code here:
salaries[salaries['Hourly Rate'].notnull()].pivot_table(index=['Department'], values=['Name'], aggfunc = ['count']).sort_values(('count','Name'),axis=0,ascending=False)
# -> STREETS & SAN has the most hourly workers
# Crosstab ????
pd.crosstab(salaries['Department'],salaries['Salary or Hourly']).sort_values('Hourly', ascending=False).head()

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [10]:
# Your code here:
hourly_sas = salaries['Hourly Rate'][(salaries['Hourly Rate'].notnull())&(salaries['Department']=='STREETS & SAN')]
display(hourly_sas.shape)

#H0: Average hourly wage in STREETS & SAN >= 35 USD (1-sided)
c = 0.95
mu_test = 35

m = np.mean(hourly_sas)
s = np.std(hourly_sas, ddof=1)
n = len(hourly_sas)
stat = (m-mu_test)/(s/np.sqrt(n))
print(stat)
# p-value 1-tailed
p = st.t.sf(abs(stat),n-1)
display(p)

# scipy function shortcut
display(st.ttest_1samp(hourly_sas,mu_test))

# compare p with c
if (p < (1-c)) & (stat > 0):
    print('H0 is rejected!')
else:
    print('H0 can not be rejected! Workers of Streets & San hourly rate is less than 35 USD')

(1862,)

-9.567447887848152


1.6689265282353859e-21

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564707717e-21)

H0 can not be rejected! Workers of Streets & San hourly rate is less than 35 USD


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [11]:
# Your code here:
hourly_all = salaries['Hourly Rate'][(salaries['Hourly Rate'].notnull())]
c = 0.95
n = len(hourly_all)
x = np.mean(hourly_all)
print('Mean:',x)
###########

# standard error with sem function:
e = st.sem(hourly_all, axis=0, ddof=1, nan_policy='omit')
print('Standard error:',e)

# confidence interval with interval function:
confidence_interval = st.t.interval(1-c, n-1, loc=x, scale=e) # df = n-1? wtf?
confidence_interval

Mean: 32.78855771628024
Standard error: 0.1352368565101596


(32.780077183391256, 32.79703824916922)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [12]:
# Your code here:
c = 0.95
n = len(salaried_police)
x = np.mean(salaried_police)
print('Mean:',x)

# standard error with sem function:
e = st.sem(salaried_police, axis=0, ddof=1, nan_policy='omit')
print('Standard error:',e)

# confidence interval with interval function:
confidence_interval = st.t.interval(1-c, n-1, loc=x, scale=e)
confidence_interval

Mean: 86486.41450313339
Standard error: 157.82445675052244


(86476.5176546444, 86496.31135162238)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:
from statsmodels.stats.proportion import proportions_ztest
#proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)

number_hourly_workers = len(salaries[salaries['Hourly Rate'].notnull()])
number_tot_employees = len(salaries)
share_hourly_workers = number_hourly_workers/number_tot_employees
print('Real share:',share_hourly_workers)

c = 0.94
proportions_ztest(number_hourly_workers, number_tot_employees, value=0.25, alternative='two-sided', prop_var=False)

# Returns:
#  zstat (float): test statistic for the z-test
#  p-value (float): p-value for the z-test

# Underlying H0: hourly workers of the City of Chicage equal 0.25 
# Conclusion: p-value very small -> reject H0 -> hourly workers differ signicatantly to 0.25
# but real share is 0.24175 -> confidence level

Real share: 0.24175029382515142


(-3.5099964213703005, 0.0004481127249057967)